In [1]:
ROOT = 'https://www.saksfifthavenue.com/Men/Apparel/T-Shirts/shop/_/N-52kmqv/Ne-6lvnb5?FOLDER%3C%3Efolder_id=2534374306652711&Nao={}'

In [2]:
OFFSETS = [0, 150, 300, 450, 600]

In [3]:
ROOT.format(OFFSETS[-1])

'https://www.saksfifthavenue.com/Men/Apparel/T-Shirts/shop/_/N-52kmqv/Ne-6lvnb5?FOLDER%3C%3Efolder_id=2534374306652711&Nao=600'

In [4]:
DATASET_ROOT_DESTINATION = "/home/rajkinra23/git/drip_vision/data/scraped_dataset/"
DATASET_TMP_DESTINATION = "/tmp/scraped_dataset/"

In [24]:
import requests
from bs4 import BeautifulSoup

# Parent directory import
import sys
sys.path.append('/home/rajkinra23/git/drip_vision/')
from models import model_api
from data.data_loaders import LABELS

In [6]:
# Headers 
agent = {"user-agent":
         "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36"}

# Send request
url = ROOT.format(0)
print(url)
resp = requests.get(url, headers=agent)
soup = BeautifulSoup(resp.content, 'html.parser')

https://www.saksfifthavenue.com/Men/Apparel/T-Shirts/shop/_/N-52kmqv/Ne-6lvnb5?FOLDER%3C%3Efolder_id=2534374306652711&Nao=0


In [7]:
import re
from bs4 import BeautifulSoup, SoupStrainer

"""
In this step, build a list of search links
"""

SHIRT_LINKS = set()

for offset in OFFSETS:
    url = ROOT.format(offset)
    resp = requests.get(url, headers=agent)
    soup = BeautifulSoup(resp.content, 'html.parser')
    for link in soup.findAll('a', attrs={'href': re.compile("/product")}):
        l = link.get('href')
        if 'product' in l:
            SHIRT_LINKS.add(l)

SHIRT_LINKS = list(SHIRT_LINKS)

In [8]:
print(len(SHIRT_LINKS))

74


In [21]:
# Mess around with getting shirt images
import json

"""
Build function to generate all of the image links.
"""

# Class to store image metadata
class ImageMetadata(object):
    def __init__(self, image_url, pid):
        self.image_url = image_url
        self.pid = pid


# Get the shirt link from the product page.
def get_image_links(shirt_link):
    # Get link
    shirt_link = f"https://www.saksfifthavenue.com/{shirt_link}"
    resp = requests.get(shirt_link, headers=agent)
    soup = BeautifulSoup(resp.content, 'html.parser')

    # Metadata container
    out = []

    # For each link, populate metadata
    for link in soup.find_all('div', {"class": "primary-image"}):
        for image in link.findAll('img'):
            product_id = image.attrs["data-adobelaunchfullsizeimageproductid"]
            image_url = image.attrs["src"]
            out.append(ImageMetadata(image_url, product_id))

    # Return metadata
    return out


"""
Let's give this a test drive.
"""
print(SHIRT_LINKS[0])
get_image_links(SHIRT_LINKS[0])


/product/theory-mattis-studio-waffle-knit-t-shirt-0400018021844.html?dwvar_0400018021844_color=IVORY


In [10]:
import os

os.listdir(".")
print(os.path.isdir("dataset"))

False


In [26]:
"""
Let's write a utility to download an image.
"""

import requests # to get image from the web
import shutil # to save it locally
import os
from PIL import Image

# Initialize model api
detector = model_api.EfficientDetAPI()
classifier = model_api.ClipAPI()

def write_image_to_disk(image_url, dst):    
    #Set up the image URL and filename
    filename = image_url.split("/")[-1]
    jpeg_name = os.path.split(dst)[-1]

    # Open the url image, set stream to True, this will return the stream content.
    r = requests.get(image_url, stream = True)

    # Check if the image was retrieved successfully
    if r.status_code == 200:
        # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
        r.raw.decode_content = True

        # Write img to tmp directory
        temp_jpeg = os.path.join(DATASET_TMP_DESTINATION, jpeg_name)
        with open(temp_jpeg, 'wb') as f:
            print(type(r.raw))
            shutil.copyfileobj(r.raw, f)
            
        # Use model API to determine if the image actually contains relevant content
        detections = detector.detect(temp_jpeg)
        if detections:
            print(classifier.rank_labels(Image.open(temp_jpeg).convert("RGB"), LABELS))
        
    else:
        print('Image Couldn\'t be retreived')
        
def write_image_set_to_disk(image_set, root=None, dry_run=False, overwrite=True):
    # Declare root for all images
    if not root:
        root = DATASET_ROOT_DESTINATION
    
    # Create the directory for the image set, if it doesn't already exist.
    image_set_root = os.path.join(root, image_set[0].pid)
    dir_exists = os.path.exists(image_set_root)
    if overwrite or not dir_exists:
        # Write image root dir.
        print("Writing images to: {}".format(image_set_root))
        if not dir_exists:
            os.mkdir(image_set_root)

        # Write images
        counter = 0
        for image_metadata in image_set:
            img = f"{image_metadata.pid}_A{counter}.jpg"
            dst = os.path.join(image_set_root, img)
            if not dry_run:
                write_image_to_disk(image_metadata.image_url, dst)
            counter += 1

        # Return that we wrote images
        return True

    # Otherwise, we skip.
    else:
        print("Images already written to: {}".format(image_set_root))
        return False
            
# Test drive it
# write_image_set_to_disk(get_image_links(SHIRT_LINKS[0]))
image_metadata = get_image_links(SHIRT_LINKS[0])
write_image_set_to_disk(image_metadata, dry_run=False)

Using checkpoint: f/home/rajkinra23/git/drip_vision/models/efficientdet_checkpoints/epoch=9-step=239949.ckpt
{'name': 'tf_efficientnetv2_l', 'backbone_name': 'tf_efficientnetv2_l', 'backbone_args': {'drop_path_rate': 0.2}, 'backbone_indices': None, 'image_size': [512, 512], 'num_classes': 1, 'min_level': 3, 'max_level': 7, 'num_levels': 5, 'num_scales': 3, 'aspect_ratios': [[1.0, 1.0], [1.4, 0.7], [0.7, 1.4]], 'anchor_scale': 4.0, 'pad_type': 'same', 'act_type': 'swish', 'norm_layer': None, 'norm_kwargs': {'eps': 0.001, 'momentum': 0.01}, 'box_class_repeats': 3, 'fpn_cell_repeats': 3, 'fpn_channels': 88, 'separable_conv': True, 'apply_resample_bn': True, 'conv_bn_relu_pattern': False, 'downsample_type': 'max', 'upsample_type': 'nearest', 'redundant_bias': True, 'head_bn_level_first': False, 'head_act_type': None, 'fpn_name': None, 'fpn_config': None, 'fpn_drop_path_rate': 0.0, 'alpha': 0.25, 'gamma': 1.5, 'label_smoothing': 0.0, 'legacy_focal': False, 'jit_loss': False, 'delta': 0.1, '

True

In [ ]:
print(links)
write_image_set_to_disk(links)

In [ ]:
# We probably want to use some off the shelf detector 

In [ ]:
"""
Execute the entire pipeline
"""

import time

def pipeline():
    for shirt_link in SHIRT_LINKS:
        image_links, product_id = get_image_links(shirt_link)
        if write_image_set_to_disk(image_links, product_id):
            time.sleep(2)
        print("Images downloaded for {}".format(shirt_link))
        
pipeline()

In [1]:
# Test model based scraping using our trained detector.

import sys
sys.path.append('/home/rajkinra23/git/drip_vision')

from models.efficient_det_model import load_model, MODEL_CHECKPOINTS_DIR
print(MODEL_CHECKPOINTS_DIR)

/home/rajkinra23/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/home/rajkinra23/git/drip_vision/models/efficientdet_checkpoints


In [2]:
model = load_model()

/home/rajkinra23/git/drip_vision/models/efficientdet_checkpoints/epoch=9-step=239949.ckpt
{'name': 'tf_efficientnetv2_l', 'backbone_name': 'tf_efficientnetv2_l', 'backbone_args': {'drop_path_rate': 0.2}, 'backbone_indices': None, 'image_size': [512, 512], 'num_classes': 1, 'min_level': 3, 'max_level': 7, 'num_levels': 5, 'num_scales': 3, 'aspect_ratios': [[1.0, 1.0], [1.4, 0.7], [0.7, 1.4]], 'anchor_scale': 4.0, 'pad_type': 'same', 'act_type': 'swish', 'norm_layer': None, 'norm_kwargs': {'eps': 0.001, 'momentum': 0.01}, 'box_class_repeats': 3, 'fpn_cell_repeats': 3, 'fpn_channels': 88, 'separable_conv': True, 'apply_resample_bn': True, 'conv_bn_relu_pattern': False, 'downsample_type': 'max', 'upsample_type': 'nearest', 'redundant_bias': True, 'head_bn_level_first': False, 'head_act_type': None, 'fpn_name': None, 'fpn_config': None, 'fpn_drop_path_rate': 0.0, 'alpha': 0.25, 'gamma': 1.5, 'label_smoothing': 0.0, 'legacy_focal': False, 'jit_loss': False, 'delta': 0.1, 'box_loss_weight': 5

In [3]:
# Util for prediction
def visualize_prediction(image):
    img = Image.open(image).convert("RGB")
    boxes, labels, confidences = model.predict([img])
    boxes = torch.tensor(boxes[0])
    labels = labels[0]
    return (boxes, labels)

In [27]:
# Scrape images using detector + CLIP for identifying the right product

print(SHIRT_LINKS[0])
shirt_link = f"https://www.saksfifthavenue.com/{SHIRT_LINKS[0]}"
resp = requests.get(shirt_link, headers=agent)
soup = BeautifulSoup(resp.content, 'html.parser')
for link in soup.find_all('div', {"class": "primary-image"}):
    for image in link.findAll('img'):
        # print(image)
        print(image.attrs["data-adobelaunchfullsizeimageproductid"])
        print(image.attrs["src"])

/product/theory-mattis-studio-waffle-knit-t-shirt-0400018021844.html?dwvar_0400018021844_color=IVORY
0400018021844
https://image.s5a.com/is/image/saks/0400018021844_IVORY?wid=480&hei=640&qlt=90&resMode=sharp2&op_usm=0.9,1.0,8,0
0400018021844
https://image.s5a.com/is/image/saks/0400018021844_IVORY_ASTL?wid=984&hei=1312&qlt=90&resMode=sharp2&op_usm=0.9,1.0,8,0
0400018021844
https://image.s5a.com/is/image/saks/0400018021844_IVORY_A1?wid=480&hei=640&qlt=90&resMode=sharp2&op_usm=0.9,1.0,8,0
0400018021844
https://image.s5a.com/is/image/saks/0400018021844_IVORY_A2?wid=480&hei=640&qlt=90&resMode=sharp2&op_usm=0.9,1.0,8,0
0400018021844
https://image.s5a.com/is/image/saks/0400018021844_IVORY_A3?wid=480&hei=640&qlt=90&resMode=sharp2&op_usm=0.9,1.0,8,0
